In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

In [ ]:
from filters import *
from masking import *
from simulCAP import *
from latencies import *
from excitation import *
from tests import *
from deconv import *

### Parameters for every signals

In [ ]:
t=ConvolutionCAPSimulatorSingleFilterModel.default_t_array()
md=SigmoidMaskingDegreeFunction(30, 2*1/15., 0.95)

plotMaskingDegreeFunc(md)

### True parameters (reference)

In [ ]:
lat0=Eggermont1976clickLatencies80dB #latencies

#Unitary response
ur0=URWang1979m
u00=ur0.u(t-2e-3) #shift
from scipy.ndimage  import gaussian_filter1d
sig=0.3e-3 #std of gaussian kernel in s
sig2=sig/(t[1]-t[0])
u0 = gaussian_filter1d(u00, sigma=sig2)
u0/=np.amax(np.abs(u0))
u0*=0.5*np.amax(np.abs(u00))

f_c = 3000
Q_10_0 = 4 #at f_c

#Excitation pattern
EPref=EP0= ExcitationPattern(lat0, f_c, 1.7, 70)

gf0= GaussianFilter.givenQ10(f_c, Q_10_0)


Latencies

In [ ]:
plotLatencies(lat0)

Unitary response

In [ ]:
pl.plot(t*1e3, u00, label='unitary response')
pl.plot(t*1e3, u0, label='ur + nPSHT (blur)',  color='orange')
pl.legend(loc='lower right')
pl.title("Unitary response")
pl.xlabel('t (ms)')

Example : excitation pattern with tone masker

In [ ]:
A=10**(45/20.)*np.sqrt(gf0.ERB())

mask=ToneSingleFilterMaskingPattern(A, f_c*0.9, gf0, md) 
EPm = ExcitationPattern.mask(EP0, mask)
with pl.style.context('default'):
    plotExcitationPattern(EPm, figsize=(8, 3))
    plotSimulCAPs2convGaussianKernel([EPm], sig=sig, secondPeak=0., figsize=(9, 7), plotGaussianKernel=False, plotmaskingpattern=False)


### Different masking settings

In [ ]:
nb_tones=8 #nb tone maskers
nb_noise=nb_tones #nb high pass noise maskers

freqs=np.linspace(0.7, 1.1, nb_tones)*f_c

#Tones
A=10**(45/20.)*np.sqrt(gf0.ERB())
#some variations
A_arr=A*10**(2*np.random.randn((nb_tones))/10.)
mc_tones = [ToneSingleFilterMaskingCondition(A_arr[i], freqs[i]) for i in range(nb_tones)]

#HP noise
Ihz=55+3*np.random.randn((nb_noise))
mc_hpnoises = [HighPassNoiseSingleFilterMaskingCondition(Ihz[i], freqs[i]) for i in range(nb_noise)]

mcs = [NoMaskingCondition()]+mc_tones+mc_hpnoises

capSimulator = ConvolutionCAPSimulatorSingleFilterModel(lat0, gf0, EP0, md, mcs, ur=u0)

plotConvCAPSimulator(capSimulator, figsize=(14,12))

### CAP signals

Add noise

In [ ]:
CAPs0=capSimulator.simulCAPs()
CAPamax=np.amax(np.abs(CAPs0))
CAPs=CAPs0+1/15*CAPamax*np.random.randn(*np.shape(CAPs0))

In [ ]:
pl.figure(figsize=(12, 5))
pl.suptitle('CAPs + noise')
m=capSimulator.m
nb_col=(m+1)//2 if m<=12 else (m+2)//3
nb_row=(m+nb_col-1)//nb_col
for i in range(m):
    ind=i+1
    pl.subplot(nb_row, nb_col, ind)
    pl.plot(t*1e3, CAPs[i])
    pl.xlabel('t (ms)')
    pl.ylim([-1.2*CAPamax, 1.2*CAPamax])
    pl.gca().get_yaxis().set_visible(False)
pl.show()

### Non-blind deconvolution

Wiener filter + low pass

In [ ]:
CAPs_f=np.zeros_like(CAPs)
window=sg.tukey(np.shape(CAPs)[1], alpha=0.2)
for i in range(capSimulator.m):
    #window signal (tuckey window)
    CAP_w=window*CAPs[i]
    CAPs_f[i]=sg.wiener(CAP_w)
    #low pass (gaussian kernel to simply for now)
    sig_f=0.2e-3 #in s
    sig_f=sig_f/(t[1]-t[0])
    CAPs_f[i]=gaussian_filter1d(CAPs_f[i], sigma=sig_f)

In [ ]:
pl.figure(figsize=(12, 5))
pl.suptitle('CAPs (ref. and filtered)')
m=capSimulator.m
nb_col=(m+1)//2 if m<=12 else (m+2)//3
nb_row=(m+nb_col-1)//nb_col
for i in range(m):
    ind=i+1
    pl.subplot(nb_row, nb_col, ind)
    pl.plot(t*1e3, CAPs0[i])
    pl.plot(t*1e3, CAPs_f[i])
    pl.xlabel('t (ms)')
    pl.ylim([-1.2*CAPamax, 1.2*CAPamax])
    pl.gca().get_yaxis().set_visible(False)
pl.show()

In [ ]:
#deconvs

#given u
u_fft=np.fft.rfft(u0)
EP_deconv=np.zeros_like(CAPs_f)
for i in range(capSimulator.m):
    CAP_fft=np.fft.rfft(CAPs_f[i])
    EP_fft=CAP_fft/u_fft
    EP_deconv[i]=np.fft.irfft(EP_fft)
    
#given EPs
EPs0=capSimulator.getEPs()
u_deconv=np.zeros_like(CAPs_f)
for i in range(capSimulator.m):
    CAP_fft=np.fft.rfft(CAPs_f[i])
    EP_fft=np.fft.rfft(EPs0[i])
    u_fft=CAP_fft/EP_fft
    u_deconv[i]=np.fft.irfft(u_fft)
    

In [ ]:
def plotMatrices(X, X_ref, title=''):
    pl.figure(figsize=(12, 5))
    pl.suptitle(title)
    m=capSimulator.m
    nb_col=(m+1)//2 if m<=12 else (m+2)//3
    nb_row=(m+nb_col-1)//nb_col
    for i in range(m):
        ind=i+1
        pl.subplot(nb_row, nb_col, ind)
        pl.plot(t*1e3, X_ref[i])
        pl.plot(t*1e3, X[i])
        pl.xlabel('t (ms)')
        pl.gca().get_yaxis().set_visible(False)
    pl.show()

In [ ]:
EPs0=capSimulator.getEPs()
plotMatrices(EP_deconv, EPs0, title='Test deconv of EPs (if u is known)')

In [ ]:
u0_mat=np.tile(u0, (capSimulator.m, 1))
plotMatrices(u_deconv, u0_mat, title='Test deconv of u (if EPs are known)')

In [ ]:
# with gradient descent (/Newton algorithm) + projs
#given u
u_fft=np.fft.rfft(u0)

EPs0=capSimulator.getEPs()
EP1=ExcitationPattern(lat0, f_c, 1.5, 50)
EP_deconv=np.tile(EP1.E(t), (capSimulator.m, 1))
nb_steps=6
alpha=np.linspace(0.6, 0.1, nb_steps)
u_fft=np.fft.rfft(u0)
CAPs_fft=np.fft.rfft(CAPs_f, axis=1)
proj_EPs=capSimulator.get_projector()
for i in range(1, nb_steps+1):
    dEP=deconv_newton_step(EP_deconv, u_fft, CAPs_fft)
    EP_deconv-=alpha[i-1]*dEP
    #proj
    EP_deconv=proj_EPs(EP_deconv)
    if i%2==0:
        pass
        #plotMatrices(EP_deconv, EPs0, title=f'Test deconv of EPs (u is known) w/ grad. descent step {i}')
plotMatrices(EP_deconv, EPs0, title=f'Test deconv of EPs (u is known) w/ grad. descent + proj')
    
    
#given EPs
nb_steps=15
alpha=np.linspace(0.6, 0.1, nb_steps)
#other EP
#capSimulator1 = ConvolutionCAPSimulatorSingleFilterModel(lat0, gf0, EP1, md, mcs, ur=u0)
#EPs1=capSimulator1.getEPs()
#EPs_fft=np.fft.rfft(EPs1, axis=1)
EPs_fft=np.fft.rfft(EPs0, axis=1)
u0_mat=np.tile(u0, (capSimulator.m, 1))
u1_mat=np.zeros_like(CAPs_f)
filter_mat = t>6e-3
filter_mat=np.tile(filter_mat, (capSimulator.m, 1))
for i in range(1, nb_steps+1):
    du=deconv_newton_step(u1_mat, EPs_fft, CAPs_fft)
    u1_mat-=alpha[i-1]*du
    #proj 1 
    u1_mat[filter_mat]=np.zeros_like(u1_mat[filter_mat])
    #proj 2
    u1_mat_mean=np.mean(u1_mat, axis=0)[None, :]
    u1_mat=np.repeat(u1_mat_mean, capSimulator.m, axis=0)
    if i%2==0:
        pass
        #plotMatrices(u1_mat, u0_mat, title=f'Test deconv of u (EPs are known) w/ grad. descent step {i}')
plotMatrices(u1_mat, u0_mat, title=f'Test deconv of u (EPs are known) w/ grad. descent + proj')

### Blind deconvolution

We still know the true parameters for latencies and tuning, but we know neither E0 nor u

First we make a guess on an excitation pattern

In [ ]:
EP1=ExcitationPattern(lat0, f_c, 1.6, 50)

pl.figure()
pl.title('Excitation pattern (without masking)')
pl.plot(t*1e3, EP0.E(t), label='truth')
pl.plot(t*1e3, 1.3*EP1.E(t), label='guess')
pl.xlabel('t (ms)')
pl.legend()
pl.show()

In [ ]:
nb_alternations=12
nb_steps=12 #nb steps for each deconvolution (gradient descent)
alpha=np.linspace(0.6, 0.02, nb_steps)

capSimulator1 = ConvolutionCAPSimulatorSingleFilterModel(lat0, gf0, EP1, md, mcs, ur=u0)
EPs1=capSimulator1.getEPs()
#ref signals
CAPs_fft=np.fft.rfft(CAPs_f, axis=1)
u0_mat=np.tile(u0, (capSimulator.m, 1))
EPs0=capSimulator.getEPs()
u1=np.zeros_like(t)
#proj
filter_mat = t>6e-3
filter_mat=np.tile(filter_mat, (capSimulator.m, 1))
def proj_u(u_mat):
    #proj 1 
    u_mat[filter_mat]=np.zeros_like(u_mat[filter_mat])
    #proj 2
    u_mat_mean=np.mean(u_mat, axis=0)[None, :]
    u_mat=np.repeat(u_mat_mean, capSimulator.m, axis=0)
    return u_mat
proj_EPs=capSimulator1.get_projector()
sigd_f=0.15e-3 #in s
sigd_f=sigd_f/(t[1]-t[0])

E_deconv, u1=blind_deconv_alternate_steps(EPs1, u1, CAPs_f, nb_alternations, nb_steps, alpha, proj_EPs, proj_u, 
                                          sigd_f, capSimulator1.maskingPatterns)


plotMatrices(u1_mat, u0_mat, title=f'Blind deconv of u w/ grad. descent + proj')
plotMatrices(EP_deconv, EPs0, title=f'Blind deconv of EPs w/ grad. descent + proj')


plotMatrices(EP_deconv/np.amax(EP_deconv), EPs0/np.amax(EPs0), title=f'Blind deconv of EPs w/ grad. descent + proj + norm')


In [ ]:
pl.figure()
pl.title('Excitation pattern (without masking)')
pl.plot(t*1e3, EP0.E(t), label='truth')
pl.plot(t*1e3, 1.7*EP1.E(t), label='guess')
pl.plot(t*1e3, EP_deconv[0]/(2*np.amax(EP_deconv[0])), label='after algo')
pl.xlabel('t (ms)')
pl.legend()
pl.show()


pl.figure()
pl.title('u (unitary response)')
pl.plot(t*1e3, u0, label='truth')
#pl.plot(t*1e3, u1_0, label='1st deconv')
pl.plot(t*1e3, u1*0.6/np.amax(u1), label='after algo')
pl.xlabel('t (ms)')
pl.legend()
pl.show()

### Estimation of latencies and tuning

Latencies

In [ ]:
#lat0=Eggermont1976clickLatencies80dB #latencies
dil_f=np.linspace(0.6, 1.4, 7) #dilation factors
lat_arr=[PowerLawLatencies.dilate(lat0, dil_fact, f_c) for dil_fact in dil_f]

pl.figure()
pl.title(f'Latencies (/dilatation factors)')
for lat, dil_fact in zip(lat_arr, dil_f):
    f=lat.f_from_t(t)
    pl.plot(t*1e3, f*1e-3, label=f'{dil_fact:.3f}')
pl.xlabel('t (ms)')
pl.xlim([0.5, 10])
pl.ylim([0, 10])
pl.ylabel('f (kHz)')
pl.legend()
pl.show()


$Q_{10}$ (at $f_c$)

In [ ]:
m_Q10=11 #odd is better
Q_10_arr=[Q_10_0*1.12**k for k in range(-(m_Q10-1)//2, (m_Q10-1)//2+1)]
print(f"true value: Q10={Q_10_0} at f_c={f_c:.0f} Hz")
for i in range(m_Q10):
    print(f"{i}: Q10={Q_10_arr[i]:.2f}")
gf_arr= [GaussianFilter.givenQ10(f_c, Q_10_arr[i]) for i in range(m_Q10)]

In [ ]:
errs=np.zeros((len(lat_arr), len(Q_10_arr)))

nb_alternations=12
nb_steps=12 #nb steps for each deconvolution (gradient descent)
alpha=np.linspace(0.6, 0.1, nb_steps)

#ref signals
u0_mat=np.tile(u0, (capSimulator.m, 1))
EPs0=capSimulator.getEPs()
#proj
filter_mat = t>6e-3
filter_mat=np.tile(filter_mat, (capSimulator.m, 1))

sigd_f=0.15e-3 #in s
sigd_f=sigd_f/(t[1]-t[0])  
def proj_u(u_mat):
    #proj 1 
    u_mat[filter_mat]=np.zeros_like(u_mat[filter_mat])
    #proj 2
    u_mat_mean=np.mean(u_mat, axis=0)[None, :]
    u_mat=np.repeat(u_mat_mean, capSimulator.m, axis=0)
    return u_mat

for i, lat in enumerate(lat_arr):
    for j, Q_10 in enumerate(Q_10_arr):
        gf_ij=gf_arr[j]
        capSimulator_ij = ConvolutionCAPSimulatorSingleFilterModel(lat, gf_ij, EP1, md, mcs) #no ur
        EPs1=capSimulator_ij.getEPs()
        uij=np.zeros_like(t)
        proj_ij=capSimulator_ij.get_projector()
        EPs_ij, uij=blind_deconv_alternate_steps(EPs1, uij, CAPs_f, nb_alternations, nb_steps, alpha, proj_ij, proj_u, 
                                                  sigd_f, capSimulator_ij.maskingPatterns)
        capSimulator_ij.set_E0(EPs_ij[0])
        capSimulator_ij.set_u(uij)
        CAPs_ij=capSimulator_ij.simulCAPs()
        errs[i][j]=np.linalg.norm(CAPs0-CAPs_ij)  #CAPs not filtered (and noise free)
#         print(i, j)
#         print(f"dilatation factor: {dil_f[i]}")
#         print(f"Q10={Q_10_arr[j]:.2f}")
#         print(f'quad. err: {errs[i][j]}')

In [ ]:
pl.figure(figsize=(8,5))
pl.title('Quad. err (dB re:noise)')
err_ref=np.linalg.norm(CAPs0-CAPs) #noise 
errs_dB=10*np.log10(errs/err_ref) #re: noise #/np.amin(errs) re: min
pl.imshow(errs_dB, vmax=5) #interpolation='bilinear')
pl.gca().set_yticks(range(len(dil_f)))
pl.gca().set_yticklabels([f'{dil_factor:.2f}' for dil_factor in dil_f])
pl.gca().set_xticks(range(len(Q_10_arr)))
pl.gca().set_xticklabels([f'{Q10:.1f}' for Q10 in Q_10_arr])
pl.xlabel('Q10')
pl.ylabel('dil. factor')
pl.colorbar()
pl.show()